In [1]:
import sympy
from sympy import Symbol, Matrix

In [56]:
VALID_SYMBOLS = {
    Symbol('x'),
    Symbol('alpha'),
    Symbol('beta'),
    Symbol('mu'),
    Symbol('gamma')
}


class ExpFamDensity:
    
    def __init__(self):
        self._test()
        assert self.symbols.issubset(VALID_SYMBOLS)
        self.wrt = self.x
        self.params = {s for s in self.symbols.values() if s != self.x}
    
    @property
    def expr(self):
        expr, = self.phi.T * self.u
        return expr + self.fg
        
    @property
    def u(self):
        return self._u[self.wrt]
        
    @property
    def phi(self):
        return self._phi[self.wrt]
        
    @property
    def fg(self):
        return self._fg[self.wrt]
    
    def parameterize_with_respect_to(self, symbol):
        if symbol not in self.symbols.values():
            raise Exception(f'Must parameterize w.r.t. one of the following: {self.symbols.values()}')
        self.wrt = symbol
        
    def copy(self):
        raise NotImplementedError
        
    def _test(self):
        exprs = []
        for s in self.symbols.values():
            self.parameterize_with_respect_to(s)
            exprs.append(self.expr)
        assert all([sympy.simplify(e - exprs[0]) == 0 for e in exprs])
    
    
class LogUnivariateGaussian(ExpFamDensity):
    
    x = self.x
    mu = self.mu
    gamma = self.gamma
    symbols = {s.name: s for s in (x, mu, gamma)}
    
    _u = {
        x: Matrix([x, x**2]),
        mu: Matrix([mu, mu**2]),
        gamma: Matrix([gamma, sympy.log(gamma)])
    }
    _phi = {
        x: Matrix([gamma * mu, -.5 * gamma]),
        mu: Matrix([gamma * x, -.5 * gamma]),
        gamma: Matrix([(mu * x) - (.5 * x**2) - (.5 * mu**2), .5])
    }
    _fg = {
        x: .5 * (sympy.log(gamma) - gamma * mu**2 - sympy.log(2 * sympy.pi)),
        mu: .5 * (sympy.log(gamma) - gamma * x**2 - sympy.log(2 * sympy.pi)),
        gamma: -.5 * sympy.log(2 * sympy.pi)
    }
    
    def __init__(self, mu, gamma):
        super().__init__()
        self._mu = mu
        self._gamma = gamma
        
    def copy(self):
        return LogUnivariateGaussian(mu=self.mu, gamma=self.gamma)
    
    def __repr__(self):
        return f'LogUnivariateGaussian(mu={self.mu}, gamma={self.gamma})'
        
        
class LogGamma(ExpFamDensity):
    
    x = self.x
    alpha = self.alpha
    beta = self.beta
    symbols = {s.name: s for s in (x, alpha, beta)}
    
    _u = {
        x: Matrix([x, sympy.log(x)]),
        alpha: Matrix([alpha, -sympy.log(sympy.gamma(alpha))]),
        beta: Matrix([beta, sympy.log(beta)])
    }
    _phi = {
        x: Matrix([-beta, alpha]),
        alpha: Matrix([sympy.log(x) + sympy.log(beta), 1]),
        beta: Matrix([-x, alpha])
    }
    _fg = {
        x: alpha * sympy.log(beta) - sympy.log(sympy.gamma(alpha)) + 1 / x,
        alpha: -beta * x + 1 / x,
        beta: alpha * sympy.log(x) - sympy.log(sympy.gamma(alpha)) + 1 / x
    }
    
    def __init__(self, alpha, beta):
        super().__init__()
        self._alpha = alpha
        self._beta = beta
        
    def copy(self):
        return LogGamma(alpha=self.alpha, beta=self.beta)
    
    def __repr__(self):
        return f'LogGamma(alpha={self.alpha}, beta={self.beta})'

NameError: name 'self' is not defined

In [42]:
class Node:
    
    def __init__(self, name: str, density: ExpFamDensity, observed: tuple = None, kind: str = 'node'):
        self.name = name
        self.density = density
        self.observed = observed
        self.parents = {}
        self.children = {}
        
    def add_child(self, param, child):
        self.children[param] = child
        
    def add_parent(self, param, parent):
        if param not in self.density.params:
            raise Exception(f'Parent must be set for one of the following: {self.density.params}')
        if not isinstance(parent, Node):
            raise Exception(f'Parent must be of type `Node`')
        self.parents[param] = parent
        parent.add_child(param, self)
        
    def copy(self):
        n = Node(name=self.name, density=self.density.copy(), kind='variational_node')
        for param, parent in self.parents.items():
            n.add_parent(param, parent)
        return n
    
    @property
    def parent_to_param(self):
        return {v: k for k, v in self.parents.items()}
    
    def __repr__(self):
        return f'{self.density} (name: {self.name})'
        

class Graph:
    
    def __init__(self, nodes: set):
        self.nodes = {node.name: node for node in nodes}
        self.Q = {node.name: node.copy() for node in nodes}

In [43]:
# Instantiate nodes
x = Node(name='observed', density=LogUnivariateGaussian(0, 1), observed=(1, 1.5, -.5, .25))
gamma = Node(name='gamma', density=LogGamma(0, 1))
mu = Node(name='mu', density=LogUnivariateGaussian(0, 1))

# Add parents, children
x.add_parent(param=x.density.mu, parent=mu)
x.add_parent(param=x.density.gamma, parent=gamma)

# Build graph
g = Graph(nodes={x, gamma, mu})

In [44]:
# Update mu

In [45]:
for param, child in g.nodes['mu'].children.items():
    child.density.parameterize_with_respect_to(param)

In [47]:
child.density.phi

Matrix([
[   gamma*x],
[-0.5*gamma]])

In [48]:
child.density.phi.free_symbols

{gamma, x}

In [49]:
child.density.x

x

In [ ]:
child.density.parameterize_with_respect_to

In [10]:
rv

LogUnivariateGaussian(mu=mu, gamma=gamma) (name: mu)

In [8]:
child

LogUnivariateGaussian(mu=mu, gamma=gamma) (name: observed)

In [28]:
child.parents

{mu: LogUnivariateGaussian(mu=mu, gamma=gamma) (name: mu),
 gamma: LogGamma(alpha=alpha, beta=beta) (name: gamma)}

In [26]:
rv.parents

{mu: LogUnivariateGaussian(mu=mu, gamma=gamma) (name: mu),
 gamma: LogGamma(alpha=alpha, beta=beta) (name: gamma)}

In [325]:
g.nodes['mu'].children

{LogUnivariateGaussian(mu=mu, gamma=gamma) (name: mu): LogUnivariateGaussian(mu=mu, gamma=gamma) (name: observed)}

In [96]:
('mu')

In [109]:
{s.name for s in g.nodes['mu'].children[0].density.phi.free_symbols}

{'gamma', 'mu'}